In [7]:
# Import the dependencies.
import pandas as pd
import gmaps
import gmaps.datasets
import requests
# Import the API key.
from config import g_key
# import the random module
import random
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
from datetime import datetime
import requests
import time

from scipy.stats import linregress # this is for linear regression

In [8]:
# store the CSV you saved created inpart one into a DataFrame
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Buala,SB,2021-12-06 21:48:54,-8.1450,159.5921,82.33,80,100,6.02
1,1,Jamestown,US,2021-12-06 21:47:57,42.0970,-79.2353,36.99,66,90,24.00
2,2,Valdivia,CL,2021-12-06 21:48:55,-39.8142,-73.2459,70.57,52,1,5.75
3,3,Victoria,HK,2021-12-06 21:47:22,22.2855,114.1577,65.32,66,6,1.01
4,4,Hilo,US,2021-12-06 21:44:53,19.7297,-155.0900,79.29,90,90,9.22


In [9]:
# get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [10]:
# configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [15]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Ask the customer to add a min and max temp value
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [29]:
# filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp)
                                   & (city_data_df['Max Temp'] >= 75)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Buala,SB,2021-12-06 21:48:54,-8.1450,159.5921,82.33,80,100,6.02
4,4,Hilo,US,2021-12-06 21:44:53,19.7297,-155.0900,79.29,90,90,9.22
6,6,San Juan,AR,2021-12-06 21:46:32,-31.5375,-68.5364,88.52,30,45,19.46
21,21,Cruzeiro Do Sul,BR,2021-12-06 21:49:01,-29.5128,-51.9853,77.52,68,25,9.86
26,26,Georgetown,MY,2021-12-06 21:47:19,5.4112,100.3354,78.89,81,20,1.99
28,28,Vaini,TO,2021-12-06 21:49:04,-21.2000,-175.2000,80.76,83,40,10.36
29,29,Matara,LK,2021-12-06 21:49:04,5.9485,80.5353,75.16,91,35,4.50
32,32,Sao Geraldo Do Araguaia,BR,2021-12-06 21:49:05,-6.4006,-48.5550,76.87,88,82,1.52
37,37,Rikitea,PF,2021-12-06 21:49:07,-23.1203,-134.9692,76.30,73,67,8.50
39,39,Atuona,PF,2021-12-06 21:49:07,-9.8000,-139.0333,79.21,67,13,20.11


In [30]:
preferred_cities_df.count()

City_ID       163
City          163
Country       163
Date          163
Lat           163
Lng           163
Max Temp      163
Humidity      163
Cloudiness    163
Wind Speed    163
dtype: int64

In [31]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Buala,SB,82.33,-8.1450,159.5921,
4,Hilo,US,79.29,19.7297,-155.0900,
6,San Juan,AR,88.52,-31.5375,-68.5364,
21,Cruzeiro Do Sul,BR,77.52,-29.5128,-51.9853,
26,Georgetown,MY,78.89,5.4112,100.3354,
28,Vaini,TO,80.76,-21.2000,-175.2000,
29,Matara,LK,75.16,5.9485,80.5353,
32,Sao Geraldo Do Araguaia,BR,76.87,-6.4006,-48.5550,
37,Rikitea,PF,76.30,-23.1203,-134.9692,
39,Atuona,PF,79.21,-9.8000,-139.0333,


In [32]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [33]:
# Import API key
from config import g_key

In [35]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # get the lat and long
    lat = row['Lat']
    lng = row['Lng']
    
    # add the lat and long to the location key for the params dict
    params['location'] = f'{lat},{lng}'
    
    # use the search term: 'lodging' and our lat and long
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make a request and get the json data from the search
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [36]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Buala,SB,82.33,-8.1450,159.5921,Maringe Lagoon Lodge
4,Hilo,US,79.29,19.7297,-155.0900,Hilo Hawaiian Hotel
6,San Juan,AR,88.52,-31.5375,-68.5364,Hotel Provincial
21,Cruzeiro Do Sul,BR,77.52,-29.5128,-51.9853,Estrela Palace Hotel
26,Georgetown,MY,78.89,5.4112,100.3354,Cititel Penang
28,Vaini,TO,80.76,-21.2000,-175.2000,Keleti Beach Resort
29,Matara,LK,75.16,5.9485,80.5353,Amaloh Boutique Resort
32,Sao Geraldo Do Araguaia,BR,76.87,-6.4006,-48.5550,Hotel Por do Sol
37,Rikitea,PF,76.30,-23.1203,-134.9692,People ThankYou
39,Atuona,PF,79.21,-9.8000,-139.0333,Villa Enata


In [40]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [41]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))